# Examen: Modulo 1

In [15]:
from libreria_modulo_1 import analysis, preprocessing, visualization 
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, asin


## 0. Carga de datos

In [16]:
# Cargar todos los datasets de la carpeta datos
users = pd.read_csv('datos/users.csv')
usercuisine = pd.read_csv('datos/usercuisine.csv')
userpayment = pd.read_csv('datos/userpayment.csv')
ratings = pd.read_csv('datos/ratings.csv')
restaurants = pd.read_csv('datos/restaurants.csv')
parking = pd.read_csv('datos/parking.csv')
cuisine = pd.read_csv('datos/cuisine.csv')
payment_methods = pd.read_csv('datos/payment_methods.csv')
hours = pd.read_csv('datos/hours.csv')

# Reemplazar '?' con NaN en todos los dataframes
dataframes = {
    'users': users,
    'usercuisine': usercuisine,
    'userpayment': userpayment,
    'ratings': ratings,
    'restaurants': restaurants,
    'parking': parking,
    'cuisine': cuisine,
    'payment_methods': payment_methods,
    'hours': hours
}

for name, df in dataframes.items():
    dataframes[name] = df.replace('?', np.nan).infer_objects(copy=False)

# Actualizar las variables originales
users = dataframes['users']
usercuisine = dataframes['usercuisine']
userpayment = dataframes['userpayment']
ratings = dataframes['ratings']
restaurants = dataframes['restaurants']
parking = dataframes['parking']
cuisine = dataframes['cuisine']
payment_methods = dataframes['payment_methods']
hours = dataframes['hours']

print("✅ Datasets cargados exitosamente:")
for name, df in dataframes.items():
    print(f"  - {name}: {df.shape[0]} filas x {df.shape[1]} columnas")

✅ Datasets cargados exitosamente:
  - users: 138 filas x 19 columnas
  - usercuisine: 330 filas x 2 columnas
  - userpayment: 177 filas x 2 columnas
  - ratings: 1161 filas x 5 columnas
  - restaurants: 130 filas x 21 columnas
  - parking: 702 filas x 2 columnas
  - cuisine: 916 filas x 2 columnas
  - payment_methods: 1314 filas x 2 columnas
  - hours: 2339 filas x 3 columnas


C:\Users\leono\AppData\Local\Temp\ipykernel_22472\1481817673.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframes[name] = df.replace('?', np.nan).infer_objects(copy=False)


## 1. Analisis de Datos

## 1.1Creacion de columnas Ingeniería de datos:

 Teniendo muy presente la unidad muestral, genere al menos 5 variables adicionales a las proporcionadas individualmente. 

In [17]:
def crear_variables_calificaciones(ratings, users, restaurants, 
                                  usercuisine, cuisine, 
                                  userpayment, payment_methods):
    """
    PASO 1: Crear las 5 variables adicionales
    """
    # 1. Dataset base
    df = ratings.copy()
    
    # 2. Unir datos
    df = df.merge(
        users[['userID', 'latitude', 'longitude', 'smoker', 
              'dress_preference', 'budget', 'ambience']],
        on='userID'
    )
    
    df = df.merge(
        restaurants[['placeID', 'latitude', 'longitude', 'smoking_area',
                    'dress_code', 'price', 'Rambience', 'alcohol']],
        on='placeID',
        suffixes=('_user', '_rest')
    )
    
    # 3. Función distancia
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        return R * 2 * asin(sqrt(a))
    
    # Variable 1: distancia_km
    df['distancia_km'] = df.apply(
        lambda row: haversine(row['latitude_user'], row['longitude_user'],
                             row['latitude_rest'], row['longitude_rest']),
        axis=1
    )
    
    # Diccionarios para búsqueda rápida
    user_cuisine_dict = usercuisine.groupby('userID')['Rcuisine'].apply(set).to_dict()
    rest_cuisine_dict = cuisine.groupby('placeID')['Rcuisine'].apply(set).to_dict()
    user_payment_dict = userpayment.groupby('userID')['Upayment'].apply(set).to_dict()
    rest_payment_dict = payment_methods.groupby('placeID')['Rpayment'].apply(set).to_dict()
    
    # Variable 2: match_cocina (0/1)
    def check_cocina_match(user_id, place_id):
        user_set = user_cuisine_dict.get(user_id, set())
        rest_set = rest_cuisine_dict.get(place_id, set())
        return 1 if len(user_set & rest_set) > 0 else 0
    
    df['match_cocina'] = df.apply(
        lambda row: check_cocina_match(row['userID'], row['placeID']), axis=1
    )
    
    # Variable 3: compat_fumador (0,1,2)
    def compat_fumador(smoker, smoking_area):
        if smoker == 'true':
            return 2 if smoking_area != 'none' else 0
        else:
            return 1
    
    df['compat_fumador'] = df.apply(
        lambda row: compat_fumador(row['smoker'], row['smoking_area']), axis=1
    )
    
    # Variable 4: payment_match_ratio (0-1)
    def payment_match_ratio(user_id, place_id):
        user_set = user_payment_dict.get(user_id, set())
        rest_set = rest_payment_dict.get(place_id, set())
        
        if len(user_set) == 0:
            return 0
        return len(user_set & rest_set) / len(user_set)
    
    df['payment_match_ratio'] = df.apply(
        lambda row: payment_match_ratio(row['userID'], row['placeID']), axis=1
    )
    
    # Variable 5: match_ambiente (0/1)
    df['match_ambiente'] = (df['ambience'] == df['Rambience']).astype(int)
    
    return df

# Aplicar función
df_enriquecido = crear_variables_calificaciones(
    ratings, users, restaurants, usercuisine, cuisine, 
    userpayment, payment_methods
)

print("✓ Variables creadas")
print(f"Dataset shape: {df_enriquecido.shape}")
print(f"Columnas: {list(df_enriquecido.columns)}")



✓ Variables creadas
Dataset shape: (1161, 23)
Columnas: ['userID', 'placeID', 'rating', 'food_rating', 'service_rating', 'latitude_user', 'longitude_user', 'smoker', 'dress_preference', 'budget', 'ambience', 'latitude_rest', 'longitude_rest', 'smoking_area', 'dress_code', 'price', 'Rambience', 'alcohol', 'distancia_km', 'match_cocina', 'compat_fumador', 'payment_match_ratio', 'match_ambiente']


In [18]:
df_enriquecido.head()

,userID,placeID,rating,food_rating,service_rating,latitude_user,longitude_user,smoker,dress_preference,budget,...,smoking_area,dress_code,price,Rambience,alcohol,distancia_km,match_cocina,compat_fumador,payment_match_ratio,match_ambiente
0,U1077,135085,2,2,2,22.156469,-100.985540,false,elegant,medium,...,not permitted,informal,medium,familiar,No_Alcohol_Served,0.695581,0,1,0.333333,0
1,U1077,135038,2,2,1,22.156469,-100.985540,false,elegant,medium,...,section,informal,medium,familiar,No_Alcohol_Served,0.805648,0,1,0.333333,0
2,U1077,132825,2,2,2,22.156469,-100.985540,false,elegant,medium,...,none,informal,low,familiar,No_Alcohol_Served,1.040306,1,1,0.333333,0
3,U1077,135060,1,2,2,22.156469,-100.985540,false,elegant,medium,...,none,informal,medium,familiar,No_Alcohol_Served,0.728008,0,1,0.333333,0
4,U1068,135104,1,1,2,23.752269,-99.168605,false,informal,low,...,not permitted,informal,medium,familiar,Full_Bar,0.081178,1,1,1.000000,0


In [19]:
df_enriquecido.describe()

,placeID,rating,food_rating,service_rating,latitude_user,longitude_user,latitude_rest,longitude_rest,distancia_km,match_cocina,compat_fumador,payment_match_ratio,match_ambiente
count,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.0
mean,134192.041344,1.199828,1.215332,1.090439,21.893060,-100.496817,21.985639,-100.608054,22.471537,0.188630,0.981912,0.785458,0.0
std,1100.916275,0.773282,0.792294,0.790844,1.308759,0.788721,1.146560,0.726277,81.836541,0.391383,0.438080,0.362993,0.0
min,132560.000000,0.000000,0.000000,0.000000,18.813348,-101.054680,18.859803,-101.028600,0.000000,0.000000,0.000000,0.000000,0.0
25%,132856.000000,1.000000,1.000000,0.000000,22.138127,-100.984268,22.143078,-100.989134,0.932018,0.000000,1.000000,0.500000,0.0
50%,135030.000000,1.000000,1.000000,1.000000,22.150891,-100.947888,22.150802,-100.977412,2.272634,0.000000,1.000000,1.000000,0.0
75%,135059.000000,2.000000,2.000000,2.000000,22.177726,-99.250000,22.156469,-100.931311,4.728010,0.000000,1.000000,1.000000,0.0
max,135109.000000,2.000000,2.000000,2.000000,23.771030,-99.067106,23.760268,-99.126506,411.620805,1.000000,2.000000,1.000000,0.0


### 1.1 Construcción de variable objetivo:
 
  Genera la variable objetivo que quiere estimar, ya sea continua o categórica. 

In [20]:
def crear_variable_objetivo(df):
    """
    PASO 2: Crear variable objetivo categórica
    """
    # Crear rating_category
    df['rating_category'] = pd.cut(
        df['rating'],
        bins=[-float('inf'), 0, 1, float('inf')],
        labels=['Bajo', 'Medio', 'Alto']
    )
    
    # Verificar distribución
    print("Distribución de rating_category:")
    print(df['rating_category'].value_counts(normalize=True).round(3))
    
    return df

df_enriquecido = crear_variable_objetivo(df_enriquecido)

Distribución de rating_category:
rating_category
Alto     0.419
Medio    0.363
Bajo     0.219
Name: proportion, dtype: float64


In [21]:
df_enriquecido.head()

,userID,placeID,rating,food_rating,service_rating,latitude_user,longitude_user,smoker,dress_preference,budget,...,dress_code,price,Rambience,alcohol,distancia_km,match_cocina,compat_fumador,payment_match_ratio,match_ambiente,rating_category
0,U1077,135085,2,2,2,22.156469,-100.985540,false,elegant,medium,...,informal,medium,familiar,No_Alcohol_Served,0.695581,0,1,0.333333,0,Alto
1,U1077,135038,2,2,1,22.156469,-100.985540,false,elegant,medium,...,informal,medium,familiar,No_Alcohol_Served,0.805648,0,1,0.333333,0,Alto
2,U1077,132825,2,2,2,22.156469,-100.985540,false,elegant,medium,...,informal,low,familiar,No_Alcohol_Served,1.040306,1,1,0.333333,0,Alto
3,U1077,135060,1,2,2,22.156469,-100.985540,false,elegant,medium,...,informal,medium,familiar,No_Alcohol_Served,0.728008,0,1,0.333333,0,Medio
4,U1068,135104,1,1,2,23.752269,-99.168605,false,informal,low,...,informal,medium,familiar,Full_Bar,0.081178,1,1,1.000000,0,Medio


### 1.3 Limpieza de datos

De acuerdo con lo visto en clase, identifique si es necesario la aplicación de los procesos que se listan a continuación: 

* Detección y remoción de valores extremos. 
* Detección y remoción de variables poco pobladas. 65% 
* Detección y tratamiento de valores ausentes. 
* Remoción de variables altamente correlacionadas. Remueva solo aquellas con correlación 1 en valor absoluto.
* Detección y remoción de variables unitarias (unarias). 


#### 1.3.1 Detección y remoción de variables poco pobladas. 65% 

In [22]:
analysis.check_data_completeness_JavierMartinezReyes(df_enriquecido)

ANÁLISIS COMPLETO DE COMPLETITUD DE DATOS
Dimensiones del DataFrame: 1161 filas x 24 columnas
Total de valores: 27,864
Total de valores nulos: 209
Porcentaje general de completitud: 99.25%

CLASIFICACIÓN DE VARIABLES:
- Categórica_Baja: 10 columnas
- Discreta: 7 columnas
- Continua: 6 columnas
- Categórica_Alta: 1 columnas

Columnas con mayor porcentaje de nulos:
- budget: 5.94% (Categórica_Baja)
- ambience: 5.0% (Categórica_Baja)
- dress_preference: 4.39% (Categórica_Baja)
- smoker: 2.67% (Categórica_Baja)
- food_rating: 0.0% (Discreta)


{'resumen_general':                 columna tipo_dato  valores_totales  valores_no_nulos  \
 0                budget    object             1161              1092   
 1              ambience    object             1161              1103   
 2      dress_preference    object             1161              1110   
 3                smoker    object             1161              1130   
 4           food_rating     int64             1161              1161   
 5                userID    object             1161              1161   
 6                rating     int64             1161              1161   
 7               placeID     int64             1161              1161   
 8        longitude_user   float64             1161              1161   
 9         latitude_user   float64             1161              1161   
 10       service_rating     int64             1161              1161   
 11        latitude_rest   float64             1161              1161   
 12       longitude_rest   float

In [23]:
analysis.completitud_datos(df_enriquecido)

budget                 0.0594
ambience               0.0500
dress_preference       0.0439
smoker                 0.0267
food_rating            0.0000
userID                 0.0000
rating                 0.0000
placeID                0.0000
longitude_user         0.0000
latitude_user          0.0000
service_rating         0.0000
latitude_rest          0.0000
longitude_rest         0.0000
smoking_area           0.0000
dress_code             0.0000
price                  0.0000
Rambience              0.0000
alcohol                0.0000
distancia_km           0.0000
match_cocina           0.0000
compat_fumador         0.0000
payment_match_ratio    0.0000
match_ambiente         0.0000
rating_category        0.0000
dtype: float64

No se eliminaron columnas debdo a que en su mayoria todas tienen un alto grado de completitud.

### 1.3.2  Detección y tratamiento de valores ausentes. 


In [31]:
datos_imputados = preprocessing.impute_missing_values(df_enriquecido, method='auto')


Valores faltantes antes de imputar:
209 en total
Columna 'smoker' (categórica): imputada con moda 'false'
Columna 'dress_preference' (categórica): imputada con moda 'no preference'
Columna 'budget' (categórica): imputada con moda 'medium'
Columna 'ambience' (categórica): imputada con moda 'family'

Valores faltantes después de imputar: 0


In [35]:
datos_imputados.sample(5)

,userID,placeID,rating,food_rating,service_rating,latitude_user,longitude_user,smoker,dress_preference,budget,...,dress_code,price,Rambience,alcohol,distancia_km,match_cocina,compat_fumador,payment_match_ratio,match_ambiente,rating_category
771,U1098,135060,2,1,2,22.182571,-100.963232,false,formal,medium,...,informal,medium,familiar,No_Alcohol_Served,3.259735,0,1,1.000000,0,Alto
773,U1046,135026,1,0,2,22.144415,-100.933097,false,no preference,medium,...,informal,high,familiar,Full_Bar,7.037418,1,1,0.000000,0,Medio
30,U1070,132613,1,1,0,23.753237,-99.166868,false,informal,low,...,informal,medium,familiar,No_Alcohol_Served,0.186114,1,1,1.000000,0,Medio
545,U1045,135064,0,0,0,22.156724,-100.984268,false,elegant,high,...,informal,high,familiar,No_Alcohol_Served,1.291769,0,1,1.000000,0,Bajo
871,U1086,132922,2,1,2,22.157281,-100.984440,false,no preference,medium,...,formal,medium,familiar,No_Alcohol_Served,0.717720,0,1,0.333333,0,Alto


### 1.3  Detección y remoción de valores extremos.


In [27]:
outliers_iqr = preprocessing.detect_outliers_iqr(datos_imputados, factor=1.5)
outliers_zscore = preprocessing.detect_outliers_zscore(datos_imputados, threshold=1.5)

In [28]:
outliers_iqr

,columna,Q1,Q3,IQR,limite_inferior,limite_superior,num_outliers,porcentaje_outliers
0,placeID,132856.000000,135059.000000,2203.000000,129551.500000,138363.500000,0,0.000000
1,rating,1.000000,2.000000,1.000000,-0.500000,3.500000,0,0.000000
2,food_rating,1.000000,2.000000,1.000000,-0.500000,3.500000,0,0.000000
3,service_rating,0.000000,2.000000,2.000000,-3.000000,5.000000,0,0.000000
4,latitude_user,22.138127,22.177726,0.039599,22.078729,22.237125,314,27.045650
5,longitude_user,-100.984268,-99.250000,1.734268,-103.585670,-96.648598,0,0.000000
6,latitude_rest,22.143078,22.156469,0.013391,22.122992,22.176555,275,23.686477
7,longitude_rest,-100.989134,-100.931311,0.057823,-101.075868,-100.844576,240,20.671835
8,distancia_km,0.932018,4.728010,3.795992,-4.761971,10.421998,96,8.268734
9,match_cocina,0.000000,0.000000,0.000000,0.000000,0.000000,219,18.863049


In [29]:
outliers_zscore

,columna,media,std,threshold,num_outliers,porcentaje_outliers
0,placeID,134192.041344,1100.916275,1.5,0,0.000000
1,rating,1.199828,0.773282,1.5,254,21.877692
2,food_rating,1.215332,0.792294,1.5,266,22.911283
3,service_rating,1.090439,0.790844,1.5,0,0.000000
4,latitude_user,21.893060,1.308759,1.5,164,14.125754
5,longitude_user,-100.496817,0.788721,1.5,304,26.184324
6,latitude_rest,21.985639,1.146560,1.5,240,20.671835
7,longitude_rest,-100.608054,0.726277,1.5,240,20.671835
8,distancia_km,22.471537,81.836541,1.5,64,5.512489
9,match_cocina,0.188630,0.391383,1.5,219,18.863049


´Debido a que son latitudes, datos de ubicacion, se considera no eeliinarlos ya que pueden proporcionar informacion al analisis´

### 1.3.1  Visualizacion de valores extremos.


In [30]:
visualization.plot_interactive_histogram(datos_imputados, column='latitude', group_by='city')

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['userID', 'placeID', 'rating', 'food_rating', 'service_rating', 'latitude_user', 'longitude_user', 'smoker', 'dress_preference', 'budget', 'ambience', 'latitude_rest', 'longitude_rest', 'smoking_area', 'dress_code', 'price', 'Rambience', 'alcohol', 'distancia_km', 'match_cocina', 'compat_fumador', 'payment_match_ratio', 'match_ambiente', 'rating_category'] but received: latitude

## 2. Analisis de datos

In [ ]:
resultado_completo = analysis.check_data_completeness_JavierMartinezReyes(datos_imputados)

ANÁLISIS COMPLETO DE COMPLETITUD DE DATOS
Dimensiones del DataFrame: 130 filas x 19 columnas
Total de valores: 2,470
Total de valores nulos: 0
Porcentaje general de completitud: 100.00%

CLASIFICACIÓN DE VARIABLES:
- Categórica_Baja: 10 columnas
- Continua: 3 columnas
- Categórica_Alta: 3 columnas
- Categórica_Media: 3 columnas

Columnas con mayor porcentaje de nulos:
- placeID: 0.0% (Continua)
- latitude: 0.0% (Continua)
- longitude: 0.0% (Continua)
- the_geom_meter: 0.0% (Categórica_Alta)
- name: 0.0% (Categórica_Alta)


In [ ]:
# Mostrar resumen general
print("📊 RESUMEN GENERAL:")
resultado_completo['resumen_general']

📊 RESUMEN GENERAL:


,columna,tipo_dato,valores_totales,valores_no_nulos,valores_nulos,porcentaje_completitud,porcentaje_nulos,valores_unicos,clasificacion
0,placeID,int64,130,130,0,100.0,0.0,130,Continua
1,latitude,float64,130,130,0,100.0,0.0,129,Continua
2,longitude,float64,130,130,0,100.0,0.0,129,Continua
3,the_geom_meter,object,130,130,0,100.0,0.0,130,Categórica_Alta
4,name,object,130,130,0,100.0,0.0,129,Categórica_Alta
5,address,object,130,130,0,100.0,0.0,99,Categórica_Alta
6,city,object,130,130,0,100.0,0.0,16,Categórica_Media
7,state,object,130,130,0,100.0,0.0,12,Categórica_Media
8,country,object,130,130,0,100.0,0.0,2,Categórica_Baja
9,zip,object,130,130,0,100.0,0.0,34,Categórica_Media


In [ ]:
# Mostrar estadísticos de dispersión
print("📈 ESTADÍSTICOS DE DISPERSIÓN (Variables Numéricas):")
resultado_completo['estadisticos_dispersion']

📈 ESTADÍSTICOS DE DISPERSIÓN (Variables Numéricas):


,columna,tipo,media,mediana,desv_std,varianza,min,max,q25,q75,rango,coef_variacion
0,placeID,Continua,134012.7846,134994.0000,1140.6446,1.301070e+06,132560.0000,135109.0000,132831.0000,135050.7500,2549.0000,0.8511
1,latitude,Continua,21.8589,22.1491,1.4968,2.240500e+00,18.8598,23.7603,22.1396,22.1601,4.9005,6.8477
2,longitude,Continua,-100.3429,-100.9568,0.8606,7.406000e-01,-101.0286,-99.1265,-100.9878,-99.2205,1.9021,-0.8576


In [ ]:
# Mostrar clasificación de variables
print("🏷️ CLASIFICACIÓN AUTOMÁTICA DE VARIABLES")
resultado_completo['clasificacion_variables'].sort_values(by='clasificacion', ascending=True)

🏷️ CLASIFICACIÓN AUTOMÁTICA DE VARIABLES


,columna,clasificacion,criterio,tipo_original,es_numerica,es_categorica
3,the_geom_meter,Categórica_Alta,Valores únicos: 130,object,False,True
4,name,Categórica_Alta,Valores únicos: 129,object,False,True
5,address,Categórica_Alta,Valores únicos: 99,object,False,True
18,other_services,Categórica_Baja,Valores únicos: 3,object,False,True
16,franchise,Categórica_Baja,Valores únicos: 2,object,False,True
15,Rambience,Categórica_Baja,Valores únicos: 2,object,False,True
14,price,Categórica_Baja,Valores únicos: 3,object,False,True
13,accessibility,Categórica_Baja,Valores únicos: 3,object,False,True
8,country,Categórica_Baja,Valores únicos: 2,object,False,True
17,area,Categórica_Baja,Valores únicos: 2,object,False,True
